<h2>Lesson subject: working with missing values</h2>

We are using scan function to provided human-guided hint while searching for area that matches to particular address.

Let's use the same code to obtain data as in lesson a, but i na one cell.

In [ ]:
#load """Domain.fsx""" 
#load """Cities\Florence.fsx""" 

open Domain

Florence.license.info |> display 

let addresses , districts, streets, summary = 
    setup Florence.data Area.Wards Limit.NoLimit // (Limit.Streets 500) 

summary |> display

let positions = addresses |> Array.map (fun x -> x.Pos) 

let indexedDistricts = districts |> Array.indexed

The Open Data Commons – Public Domain Dedication and Licence
(http://www.opendatacommons.org/licenses/pddl/1.0/)

data summary (without limits) -> 
 areas: 129, streets: 2214, addressess: 108618 
 addresses per street (median/mean/std-dev): 27.0/49.0596206/81.04981685

Now we need to adapt our <i>hintSearch</i> human-driven algorithm slightly.
To mark computation as avoid or not we should  appropriate annotation. We could use true/false for that but let's use more descriptive type:

In [ ]:
let hintSearch lastHitAreaIndex currentPosition  =
    let hintArea = indexedDistricts.[lastHitAreaIndex]
    let find = snd >> isPointWithinFeature currentPosition

    if find hintArea then lastHitAreaIndex
    else indexedDistricts |> Array.findIndex find

let results = scan hintSearch positions 0


In [ ]:
let hintSearch lastHitAreaIndex currentPosition =

    let find = snd >> isPointWithinFeature currentPosition
    let searchAll () = indexedDistricts |> Array.tryFindIndex find

    match lastHitAreaIndex with
    | Some hint when find indexedDistricts.[hint]-> Some hint
    | _ -> searchAll ()
    

let results = scan hintSearch positions None 

Remember that we no longer have sequential processing so the addresses are not sorted as they use to be. In order to make them usable we have to return index as well.

In [ ]:
// missing values
results |> Array.filter Option.isNone |> Array.length

103

In [ ]:
// matched values
results |> Array.choose id |> Array.length

108516

In [ ]:
results |> Array.choose id |> Array.take 10

index,value
0,9
1,9
2,9
3,9
4,9
5,9
6,9
7,9
8,9
9,9
